# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

### Инструкция по выполнению проекта
1. Загрузите и подготовьте данные. Поясните порядок действий.
2. Обучите и проверьте модель для каждого региона:
    - Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.
    - Обучите модель и сделайте предсказания на валидационной выборке.
    - Сохраните предсказания и правильные ответы на валидационной выборке.
    - Напечатайте на экране средний запас предсказанного сырья и RMSE модели.
    - Проанализируйте результаты.
3. Подготовьтесь к расчёту прибыли:
    - Все ключевые значения для расчётов сохраните в отдельных переменных.
    - Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе. 
    - Напишите выводы по этапу подготовки расчёта прибыли.
4. Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
    - Выберите скважины с максимальными значениями предсказаний. 
    - Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.
    - Рассчитайте прибыль для полученного объёма сырья.
5. Посчитайте риски и прибыль для каждого региона:
    - Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
    - Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
    - Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.

### Описание данных
Данные геологоразведки трёх регионов находятся в файлах: 
1. /datasets/geo_data_0.csv.
2. /datasets/geo_data_1.csv.
3. /datasets/geo_data_2.csv.

1. id — уникальный идентификатор скважины;
2. f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
3. product — объём запасов в скважине (тыс. баррелей).

### Условия задачи:
1. Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
2. При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
3. Бюджет на разработку скважин в регионе — 10 млрд рублей.
4. При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
5. После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
6. Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
from sklearn.preprocessing import StandardScaler

In [2]:
#Считываем данные из csv-файла в датафрейм
try:
    data_0 = pd.read_csv('/datasets/geo_data_0.csv')
except:
    data_0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
    
try:
    data_1 = pd.read_csv('/datasets/geo_data_1.csv')
except:
    data_1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
    
try:
    data_2 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    data_2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')

In [3]:
#Функция получения первичной информации о датафрейме
def studying_the_data (data):
    print ('------------- Первые 5 строк ------------')
    display(data.head())
    print('')
    print('')
    print ('------------- Типы данных ------------')
    print (data.info())
    print('')
    print('')
    print ('------------- Пропуски ------------')
    count = 0
    for element in data.columns:
        if data[element].isna().sum() > 0: 
            print(element, ' - ', data[element].isna().sum(), 'пропусков')
            count = +1
        if count == 0:
            print(element, 'Пропусков НЕТ')
    print('')
    print('')
    print ('------------- Дубликаты ------------')
    if data.duplicated().sum() > 0:
        print('Дубликатов: ', data.duplicated().sum())
    else:
        print('Дубликатов НЕТ')

#Проверим 1й регион        
studying_the_data (data_0)

------------- Первые 5 строк ------------


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647




------------- Типы данных ------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


------------- Пропуски ------------
id Пропусков НЕТ
f0 Пропусков НЕТ
f1 Пропусков НЕТ
f2 Пропусков НЕТ
product Пропусков НЕТ


------------- Дубликаты ------------
Дубликатов НЕТ


In [4]:
#Проверим 2й регион
studying_the_data (data_1)

------------- Первые 5 строк ------------


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305




------------- Типы данных ------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


------------- Пропуски ------------
id Пропусков НЕТ
f0 Пропусков НЕТ
f1 Пропусков НЕТ
f2 Пропусков НЕТ
product Пропусков НЕТ


------------- Дубликаты ------------
Дубликатов НЕТ


In [5]:
#Проверим 3й регион
studying_the_data (data_2)

------------- Первые 5 строк ------------


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746




------------- Типы данных ------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


------------- Пропуски ------------
id Пропусков НЕТ
f0 Пропусков НЕТ
f1 Пропусков НЕТ
f2 Пропусков НЕТ
product Пропусков НЕТ


------------- Дубликаты ------------
Дубликатов НЕТ


In [6]:
#Удалим столбцы с id
data_0 = data_0.drop(columns=['id'])
data_1 = data_1.drop(columns=['id'])
data_2 = data_2.drop(columns=['id'])

### Вывод
Данные подготовили.

## Обучение и проверка модели

In [7]:
import warnings
warnings.filterwarnings('ignore')

#Для этого сначала выделим целевой признак и оставшиеся признаки и сохраним их в переменные
target_0 = data_0['product']
features_0 = data_0.drop('product', axis=1)

#Разделим данные на обучающую и валидационную выборки
features_0_train, features_0_valid, target_0_train, target_0_valid = train_test_split(
    features_0, target_0, test_size=0.25, random_state=12345) # отделим 25% данных для валидационной выборки

numeric_0 = ['f0', 'f1', 'f2']

scaler = StandardScaler()
scaler.fit(features_0_train[numeric_0])
features_0_train[numeric_0] = scaler.transform(features_0_train[numeric_0])
features_0_valid[numeric_0] = scaler.transform(features_0_valid[numeric_0]) 

print(features_0_train.shape)
print(target_0_train.shape[0])
print(features_0_valid.shape)
print(target_0_valid.shape[0])

#Чтобы предупреждение не появлялось
pd.options.mode.chained_assignment = None

print(features_0_train.head())

(75000, 3)
75000
(25000, 3)
25000
             f0        f1        f2
27212 -0.544828  1.390264 -0.094959
7866   1.455912 -0.480422  1.209567
62041  0.260460  0.825069 -0.204865
70185 -1.837105  0.010321 -0.147634
82230 -1.299243  0.987558  1.273181


In [8]:
#Для этого сначала выделим целевой признак и оставшиеся признаки и сохраним их в переменные
target_1 = data_1['product']
features_1 = data_1.drop('product', axis=1)

#Разделим данные на обучающую и валидационную выборки
features_1_train, features_1_valid, target_1_train, target_1_valid = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345) # отделим 25% данных для валидационной выборки

numeric_1 = ['f0', 'f1', 'f2']

scaler = StandardScaler()
scaler.fit(features_1_train[numeric_1])
features_1_train[numeric_1] = scaler.transform(features_1_train[numeric_1])
features_1_valid[numeric_1] = scaler.transform(features_1_valid[numeric_1]) 

print(features_1_train.shape)
print(target_1_train.shape[0])
print(features_1_valid.shape)
print(target_1_valid.shape[0])

#Чтобы предупреждение не появлялось
pd.options.mode.chained_assignment = None

print(features_0_train.head())

(75000, 3)
75000
(25000, 3)
25000
             f0        f1        f2
27212 -0.544828  1.390264 -0.094959
7866   1.455912 -0.480422  1.209567
62041  0.260460  0.825069 -0.204865
70185 -1.837105  0.010321 -0.147634
82230 -1.299243  0.987558  1.273181


In [9]:
#Для этого сначала выделим целевой признак и оставшиеся признаки и сохраним их в переменные
target_2 = data_2['product']
features_2 = data_2.drop('product', axis=1)

#Разделим данные на обучающую и валидационную выборки
features_2_train, features_2_valid, target_2_train, target_2_valid = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345) # отделим 25% данных для валидационной выборки

numeric_2 = ['f0', 'f1', 'f2']

scaler = StandardScaler()
scaler.fit(features_2_train[numeric_2])
features_2_train[numeric_2] = scaler.transform(features_2_train[numeric_2])
features_2_valid[numeric_2] = scaler.transform(features_2_valid[numeric_2]) 

print(features_2_train.shape)
print(target_2_train.shape[0])
print(features_2_valid.shape)
print(target_2_valid.shape[0])

#Чтобы предупреждение не появлялось
pd.options.mode.chained_assignment = None

print(features_0_train.head())

(75000, 3)
75000
(25000, 3)
25000
             f0        f1        f2
27212 -0.544828  1.390264 -0.094959
7866   1.455912 -0.480422  1.209567
62041  0.260460  0.825069 -0.204865
70185 -1.837105  0.010321 -0.147634
82230 -1.299243  0.987558  1.273181


In [10]:
#Обучим модель и сделаем предсказания на валидационной выборке
model = LinearRegression() # инициализируйте модель LinearRegression
model.fit(features_0_train, target_0_train) # обучите модель на тренировочной выборке
predictions_0_valid = model.predict(features_0_valid) # получите предсказания модели на валидационной выборке

rmse_0 = mean_squared_error(target_0_valid, predictions_0_valid) ** 0.5 # посчитаем значение метрики RMSE на валидационной выборке
print("RMSE модели линейной регрессии на валидационной выборке:", rmse_0)

mean_0 = predictions_0_valid.mean()
print("Средний запас предсказанного сырья:", mean_0)

RMSE модели линейной регрессии на валидационной выборке: 37.5794217150813
Средний запас предсказанного сырья: 92.59256778438035


In [11]:
#Обучим модель и сделаем предсказания на валидационной выборке
model = LinearRegression() # инициализируйте модель LinearRegression
model.fit(features_1_train, target_1_train) # обучите модель на тренировочной выборке
predictions_1_valid = model.predict(features_1_valid) # получите предсказания модели на валидационной выборке

rmse_1 = mean_squared_error(target_1_valid, predictions_1_valid) ** 0.5 # посчитаем значение метрики RMSE на валидационной выборке
print("RMSE модели линейной регрессии на валидационной выборке:", rmse_1)

mean_1 = predictions_1_valid.mean()
print("Средний запас предсказанного сырья:", mean_1)

RMSE модели линейной регрессии на валидационной выборке: 0.893099286775617
Средний запас предсказанного сырья: 68.728546895446


In [12]:
#Обучим модель и сделаем предсказания на валидационной выборке
model = LinearRegression() # инициализируйте модель LinearRegression
model.fit(features_2_train, target_2_train) # обучите модель на тренировочной выборке
predictions_2_valid = model.predict(features_2_valid) # получите предсказания модели на валидационной выборке

rmse_2 = mean_squared_error(target_2_valid, predictions_2_valid) ** 0.5 # посчитаем значение метрики RMSE на валидационной выборке
print("RMSE модели линейной регрессии на валидационной выборке:", rmse_2)

mean_2 = predictions_2_valid.mean()
print("Средний запас предсказанного сырья:", mean_2)

RMSE модели линейной регрессии на валидационной выборке: 40.02970873393434
Средний запас предсказанного сырья: 94.96504596800489


### Вывод
Первый регион:
RMSE модели линейной регрессии на валидационной выборке: 37.5794217150813
Средний запас предсказанного сырья: 92.59256778438035

Второй регион: 
RMSE модели линейной регрессии на валидационной выборке: 0.893099286775617
Средний запас предсказанного сырья: 68.728546895446

Третий регион:
RMSE модели линейной регрессии на валидационной выборке: 40.02970873393434
Средний запас предсказанного сырья: 94.96504596800489

Средний запас предсказанного сырья у третьего региона. RMSE (корень из средней квадратичной ошибки) меньше всего у второго региона, это означает, что модель лучше соответствует данным.

## Подготовка к расчёту прибыли

In [13]:
#При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
#Бюджет на разработку скважин в регионе — 10 млрд рублей.
count_500 = 500
count_200 = 200
budget = 10000000000 # общий бюджет
budget_well = budget / count_200 # бюджет на скважину

#При нынешних ценах один баррель сырья приносит 450 рублей дохода. 
#Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
income = 450000
volume = budget_well / income # достаточный объём сырья для безубыточной разработки новой скважины.
print('Достаточный объём сырья для безубыточной разработки новой скважины, тыс. бар.:', volume)

#После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. 
#Среди них выбирают регион с наибольшей средней прибылью.

#Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
#Выберем скважины с максимальными значениями предсказаний. 
#Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.
#Рассчитаем прибыль для полученного объёма сырья.

def profit_well(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return income * selected.sum() - budget

Достаточный объём сырья для безубыточной разработки новой скважины, тыс. бар.: 111.11111111111111


### Вывод
Во всех трех регионах запас меньше, чем достаточный объём сырья для безубыточной разработки новой скважины в 111 тыс. бар.

Первая скважина на 18 тыс.бар. Вторая скважина на 42 тыс. бар. Третья скважина на 16 тыс. бар.

## Расчёт прибыли и рисков 

In [14]:
state = RandomState(12345)

predictions_0_valid = pd.Series(predictions_0_valid)
target_0_valid.reset_index(drop=True, inplace=True)
values = []

for i in range(1000):
    
    target_0_subsample = target_0_valid.sample(n=count_500, replace=True, random_state=state)
    probs_0_subsample = predictions_0_valid[target_0_subsample.index]
    values.append(profit_well(target_0_subsample, probs_0_subsample, count_200))

values = pd.Series(values)
lower = int(values.quantile(0.025))
upper = int(values.quantile(0.975))
risk = (values < 0).mean() * 100
mean = int(values.mean())

print("Средняя выручка, руб:", mean)
print("95%-й доверительный интервал, руб:", lower, upper)
print("Риск убытков, %:", risk)

Средняя выручка, руб: 425938526
95%-й доверительный интервал, руб: -102090094 947976353
Риск убытков, %: 6.0


In [15]:
state = RandomState(12345)

predictions_1_valid = pd.Series(predictions_1_valid)
target_1_valid.reset_index(drop=True, inplace=True)
values = []

for i in range(1000):
    
    target_1_subsample = target_1_valid.sample(n=count_500, replace=True, random_state=state)
    probs_1_subsample = predictions_1_valid[target_1_subsample.index]
    values.append(profit_well(target_1_subsample, probs_1_subsample, count_200))

values = pd.Series(values)
lower = int(values.quantile(0.025))
upper = int(values.quantile(0.975))
risk = (values < 0).mean() * 100
mean = int(values.mean())

print("Средняя выручка, руб:", mean)
print("95%-й доверительный интервал, руб:", lower, upper)
print("Риск убытков, %:", risk)

Средняя выручка, руб: 515222773
95%-й доверительный интервал, руб: 68873225 931547591
Риск убытков, %: 1.0


In [16]:
state = RandomState(12345)

predictions_2_valid = pd.Series(predictions_2_valid)
target_2_valid.reset_index(drop=True, inplace=True)
values = []

for i in range(1000):
    
    target_2_subsample = target_2_valid.sample(n=count_500, replace=True, random_state=state)
    probs_2_subsample = predictions_2_valid[target_2_subsample.index]
    values.append(profit_well(target_2_subsample, probs_2_subsample, count_200))

values = pd.Series(values)
lower = int(values.quantile(0.025))
upper = int(values.quantile(0.975))
risk = (values < 0).mean() * 100
mean = int(values.mean())

print("Средняя выручка, руб:", mean)
print("95%-й доверительный интервал, руб:", lower, upper)
print("Риск убытков, %:", risk)

Средняя выручка, руб: 435008362
95%-й доверительный интервал, руб: -128880547 969706954
Риск убытков, %: 6.4


### Вывод
Первая скважина:
Средняя выручка, руб: 425938526
95%-й доверительный интервал, руб: -102090094 947976353
Риск убытков, %: 6.0

Вторая скважина:
Средняя выручка, руб: 515222773
95%-й доверительный интервал, руб: 68873225 931547591
Риск убытков, %: 1.0

Третья скважина:
Средняя выручка, руб: 435008362
95%-й доверительный интервал, руб: -128880547 969706954
Риск убытков, %: 6.4

После оценки рисков вероятность убытков меньше 2.5% только у второго региона со значением 1,0%.

Следовательно предложить можно только второй регион со средней выручкой 515 222 773 руб.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован